# Data to ChatGPT

### technical settings : import libs. and create main tables

In [ ]:
# import parameters for sql queries
from params import params

# GCP : settings
key_path = "../secrets/gcp/keybq.json"   

# BigQuery : import and settings
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
client_bigquery = bigquery.Client()

# import openai and settings libraries
import os
import openai
import sys
sys.path.insert(0, '../secrets/openai')
import openai_credentials

openai.organization = openai_credentials.openai_organization_code
openai.api_key = openai_credentials.OPENAI_API_KEY

# other functions import
from datetime import datetime
import pandas as pd

In [ ]:
# create table for chatGPT answers on bigquery

dataset_name = params['tgt_dataset_name'] # Extract dataset from params dictionary
table_name = params['interpretation_KPI_table_name'] # Extract table names from params dictionary

# Define the table schema
schema = [
    bigquery.SchemaField("date_time", "DateTime"),
    bigquery.SchemaField("analysis_name", "STRING"),
    bigquery.SchemaField("prompt", "STRING"),
    bigquery.SchemaField("response", "STRING"),
    
]

table_ref = client_bigquery.dataset(dataset_name).table(table_name) # Define the table reference
table = bigquery.Table(table_ref, schema=schema) # Define the table definition
table = client_bigquery.create_table(table, exists_ok=True) # Create or replace the table


### business settings : define roles and main business guidliness for chatGPT

In [ ]:
chatGPT_role = '''Act as a data analyst'''
business_owner = '''vocal coach entrepreneur'''
main_business_guidliness = '''Remember that in july and august there are less lessons because of summer holidays.'''

### organisation settings : define rules for chatGPT

In [ ]:
main_anwser_rules_language = """Your answer should be wirtten in english."""
main_anwser_rules_nb_of_words = """Your answer should be in 200 words max."""

### extract data for analysis

In [ ]:
# extract PI metrics and add columns description

PI_description = '''First column are months and second column are number of lessons.''' # performance indicator description

# extract a table from a dataset
query_template = '''

SELECT
    year_month
    ,sum(num_lessons) as lesson_number
FROM
    `{tgt_dataset_name}.{tgt_latest_lessons_analytics_table_name}`
WHERE 
    year_month != '2023-05' -- remove 2023-05 because it is not a full month
GROUP BY
    year_month
ORDER BY
    year_month DESC
''' 

query = query_template.format(**params) # render the query string with the parameter values
query_job = client_bigquery.query(query) # execute the query
results_client_bigquery = query_job.to_dataframe() # get the results as a pandas dataframe 
results_str = results_client_bigquery.to_string(index=False) # have from dataframe a string from results_client_bigquery

nb_lessons_metrics_description = PI_description + "\n\n" + results_str # put a question for ChatGPT and data together
print(nb_lessons_metrics_description)

In [ ]:
# extract PI metrics and add columns description

PI_description = ''' First column are months and second column are revenues.''' # performance indicator description

# extract a table from a dataset
query_template = '''

SELECT
    year_month
    ,sum(num_lessons * lesson_price) as revenues
FROM
    `{tgt_dataset_name}.{tgt_latest_lessons_analytics_table_name}`
WHERE 
    year_month != '2023-05' -- remove 2023-05 because it is not a full month
GROUP BY
    year_month
ORDER BY
    year_month DESC
''' 

query = query_template.format(**params) # render the query string with the parameter values
query_job = client_bigquery.query(query) # execute the query
results_client_bigquery = query_job.to_dataframe() # get the results as a pandas dataframe 
results_str = results_client_bigquery.to_string(index=False) # have from dataframe a string from results_client_bigquery

revenues_metrics_description = PI_description + "\n\n" + results_str # put a question for ChatGPT and data together
#print(revenues_metrics_description)

### analyse business issue #1 : nb of lessons

In [ ]:
#### define the business problematics for chatGPT and anwser rules

analysis_name = 'evolution_of_lessons_number'

business_issue = """understand the evolution of his students' lessons.
He is asking himself the folliwng questions :
What do you think about recent months evolution ?
What do you think about long terme evolution 2020 - 2023?
Dont write any recommendations, just describe what you see."""

# create a prompt to talk to chatGPT

# consolidate the prompt to send to chatGPT
prompt_for_chatGPT = (
    chatGPT_role + " for " + business_owner + "." + "\n\n"
    + business_owner + " wants you to " + business_issue + "\n\n"
    + nb_lessons_metrics_description + "\n\n"  
    + main_business_guidliness + "\n\n"
    + main_anwser_rules_language + "\n\n"
    + main_anwser_rules_nb_of_words + "\n\n"
)

print(prompt_for_chatGPT)


In [ ]:
# ask chatGPT with a prompt and save the response to the database

# fetch response from chatGPT for a promt
response_from_chatGPT_with_meta_data = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt_for_chatGPT}
  ]
)
response_from_chatGPT = response_from_chatGPT_with_meta_data.choices[0].message.get('content')

# create table with answers, graph name, answer date and time

now = datetime.today() # now = date and time

df_response = pd.DataFrame({
    'date_time': [now],
    'analysis_name': [analysis_name],
    'prompt': [prompt_for_chatGPT],
    'response': [response_from_chatGPT]
}) # create a dataframe with current date and time, response, response name

# Save the DataFrame to BigQuery table
interpretation_KPI_table_name_variable = f"{params['tgt_dataset_name']}.{params['interpretation_KPI_table_name']}"

# Define the BigQuery schema
schema = [
    bigquery.SchemaField("date_time", "DATETIME"),
    bigquery.SchemaField("analysis_name", "STRING"),
    bigquery.SchemaField("prompt", "STRING"),
    bigquery.SchemaField("response", "STRING")
]

# Configure the job
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_APPEND"
)

# Insert the data from the Pandas DataFrame into BigQuery
client_bigquery.load_table_from_dataframe(
    df_response,
    interpretation_KPI_table_name_variable,
    job_config=job_config
).result()

### analyse business issue #2 : nb of lessons + revenues

In [ ]:
# define the business problematics for chatGPT and anwser rules

analysis_name = 'evolution_lessons_number_revenues_and_prices'

business_issue = """understand the evolution and correlation between revenues and nb of lessons ?
Don't write any recommendations, just describe the evolution and correlations."""


#create a prompt to talk to chatGPT

# consolidate the prompt
prompt_for_chatGPT = (
    chatGPT_role + " for " + business_owner + "." + "\n\n"

    + business_owner + " wants you to " + business_issue + "\n\n"
    
    + main_business_guidliness + "\n\n"

    + nb_lessons_metrics_description + "\n\n"
    + revenues_metrics_description + "\n\n"
    # + average_price_metrics_description + "\n\n"

    + main_anwser_rules_language + "\n\n"
    + main_anwser_rules_nb_of_words + "\n\n"
)

# print(prompt_for_chatGPT)

# ask chatGPT with a prompt and save the response to the database

# fetch response from chatGPT for a promt
response_from_chatGPT_with_meta_data = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt_for_chatGPT}
  ]
)
response_from_chatGPT = response_from_chatGPT_with_meta_data.choices[0].message.get('content')

# create table with answers, graph name, answer date and time

now = datetime.today() # now = date and time

df_response = pd.DataFrame({
    'date_time': [now],
    'analysis_name': [analysis_name],
    'prompt': [prompt_for_chatGPT],
    'response': [response_from_chatGPT]
}) # create a dataframe with current date and time, response, response name

# Save the DataFrame to BigQuery table
interpretation_KPI_table_name_variable = f"{params['tgt_dataset_name']}.{params['interpretation_KPI_table_name']}"

# Define the BigQuery schema
schema = [
    bigquery.SchemaField("date_time", "DATETIME"),
    bigquery.SchemaField("analysis_name", "STRING"),
    bigquery.SchemaField("prompt", "STRING"),
    bigquery.SchemaField("response", "STRING")
]

# Configure the job
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_APPEND"
)

# Insert the data from the Pandas DataFrame into BigQuery
client_bigquery.load_table_from_dataframe(
    df_response,
    interpretation_KPI_table_name_variable,
    job_config=job_config
).result()

### analyse business issue #3 : nb of lessons + revenues (recommendation)

In [ ]:
#### define the business problematics for chatGPT and anwser rules

analysis_name = 'recommendation_to_increase_revenue_without_increasing_nb_of_lessons'

business_issue = """understand what methodes are there to increase the revenue without incresing the number of individual lessons ? 
For examples organise group lessons ?
Make a list of bullet points with recommendations.
"""
# create a prompt to talk to chatGPT

# consolidate the prompt
prompt_for_chatGPT = (
    chatGPT_role + " for " + business_owner + "." + "\n\n"

    + business_owner + " wants you to " + business_issue + "\n\n"
    
    + main_business_guidliness + "\n\n"

    + nb_lessons_metrics_description + "\n\n"
    + revenues_metrics_description + "\n\n"
    # + average_price_metrics_description + "\n\n"

    + main_anwser_rules_language + "\n\n"
    + main_anwser_rules_nb_of_words + "\n\n"
)

# print(prompt_for_chatGPT)

# ask chatGPT with a prompt and save the response to the database

# fetch response from chatGPT for a promt
response_from_chatGPT_with_meta_data = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt_for_chatGPT}
  ]
)
response_from_chatGPT = response_from_chatGPT_with_meta_data.choices[0].message.get('content')

# create table with answers, graph name, answer date and time

now = datetime.today() # now = date and time

df_response = pd.DataFrame({
    'date_time': [now],
    'analysis_name': [analysis_name],
    'prompt': [prompt_for_chatGPT],
    'response': [response_from_chatGPT]
}) # create a dataframe with current date and time, response, response name

# Save the DataFrame to BigQuery table
interpretation_KPI_table_name_variable = f"{params['tgt_dataset_name']}.{params['interpretation_KPI_table_name']}"

# Define the BigQuery schema
schema = [
    bigquery.SchemaField("date_time", "DATETIME"),
    bigquery.SchemaField("analysis_name", "STRING"),
    bigquery.SchemaField("prompt", "STRING"),
    bigquery.SchemaField("response", "STRING")
]

# Configure the job
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_APPEND"
)

# Insert the data from the Pandas DataFrame into BigQuery
client_bigquery.load_table_from_dataframe(
    df_response,
    interpretation_KPI_table_name_variable,
    job_config=job_config
).result()